# <img src="../img/divvy_logo.svg" class="img-header"> tutorial

## Compute packages

When you start `divvy`, you may provide a configuration file that specifies one or more *compute packages*. A compute package is just a set of a variables that contains information needed to run a job, such as a job submission template, the command that you use to submit a job (*e.g.* `sbatch` or `qsub`), and any other variables needed to fill the template (*e.g.* `partition` or `account`). You can find out [how to write your own divvy config file](../configuration), but for this tutorial, we'll just use the default.

Start by importing `divvy`, and then create a new `ComputingConfiguration` object. If you provide no arguments, you'll just get a few default packages:

In [1]:
import divvy
dcc = divvy.ComputingConfiguration()

No local config file was provided
No global config file was provided in environment variable(s): ['DIVCFG', 'PEPENV']
Using default config file.
Loading divvy config file: /home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/default_compute_settings.yaml
Available packages: set(['default', 'local', 'slurm'])
Activating compute package 'default'


This loads up the default compute package, and we see that there are a few other packages available. We can explore the compute settings in the loaded (`default`) package like this: 

In [2]:
dcc.compute

{'submission_command': 'sh', 'submission_template': '/home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/localhost_template.sub'}

Here you can see that a *compute package* is really a simple thing. In this case, it's just 2 key-value pairs. The `submission_template` key is a path to a template file, with these contents: 



In [3]:
with open(dcc.compute.submission_template) as f:
    print(f.read())

#!/bin/bash

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

{CODE} | tee {LOGFILE}



We can populate this simple template by passing values for the `{VARIABLE}` text in the template:

In [4]:
dcc.write_script("test_local.sub", {"code": "run-this-command", "logfile": "logfile.txt"})

Writing script to /home/nsheff/code/divvy/docs_jupyter/test_local.sub


'test_local.sub'

Now let's look at the contents of our populated template:

In [5]:
with open("test_local.sub") as f:
    print(f.read())

#!/bin/bash

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

run-this-command | tee logfile.txt



The `submission_command` contains the shell instruction that would be used to submit this populated template; in this case, it's simply `sh` to run this script in the console. We can activate a different *compute_package* like this: 

In [7]:
dcc.activate_package("slurm")

Activating compute package 'slurm'


True

It returns 'True' to indicate that the activation has been successful. This will change our settings. Let's inspect the new package:

In [8]:
dcc.compute

{'submission_command': 'sbatch', 'logfile': 'logfile.txt', 'submission_template': '/home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/slurm_template.sub', 'code': 'run-this-command'}

Now that we've activated the package of interest, we can use those variables to populate a submission template to create a submission script. Use the `write_script` function:

In [9]:
s = dcc.write_script("test_script.sub")

Writing script to /home/nsheff/code/divvy/docs_jupyter/test_script.sub


This function opens the template specified by the `submission_template` variable in the compute package, and then populates any template variables with values from the compute package. In this case, our compute package didn't have *all* relevant variables and so the template is returned with a few variables left unpopulated:

In [10]:
with open("test_script.sub") as f:
    print(f.read())

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='logfile.txt'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='{TIME}'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

run-this-command



In addition to using the compute package variables, we can also pass some extra variables along to the the `write_script` function. Let's pass a value for the `code` variable and see how this changes the submission script output:

In [7]:
s = dcc.write_script("test_script.sub", {"code":"yellow"})

Writing script to /home/nsheff/code/divvy/docs_jupyter/test_script.sub


Here's the output. Notice that the `{CODE}` variable has been replaced with the word `yellow`:

In [8]:
with open("test_script.sub") as f:
    print(f.read())

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='{LOGFILE}'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='{TIME}'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

yellow



## Using a priority list of variables

Now, you can also pass more than one `Dict` object, in priority order, by just passing a list. Here, we'll pass 2 dicts, and any values in the 2nd one will override values in the first:

In [9]:
s = dcc.write_script("test_script.sub", [{"code":"yellow", "time": "now"}, {"code": "red"}])

Writing script to /home/nsheff/code/divvy/docs_jupyter/test_script.sub


In [10]:
with open("test_script.sub") as f:
    print(f.read())

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='{LOGFILE}'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='now'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

red

